In [4]:
import os
import glob
import numpy as np

#### count reads number 

In [ ]:
nohup bedtools multicov \
-bams /Data_Resource/m6A-seq/com_fetus_vs_adult/gvcf/adult_data/ip/CRR042308/CRR042308-uniq.bam \
-bams /Data_Resource/m6A-seq/com_fetus_vs_adult/gvcf/adult_data/ip/CRR042306/CRR042306-uniq.bam \
-bams /Data_Resource/m6A-seq/com_fetus_vs_adult/gvcf/adult_data/input/CRR042309/CRR042309-uniq.bam \
-bams /Data_Resource/m6A-seq/com_fetus_vs_adult/gvcf/adult_data/input/CRR042307/CRR042307-uniq.bam \
-bed right_region_100bp.bed > stomach_right.count &

In [2]:
os.chdir("/Data_Resource/m6A-seq/com_fetus_vs_adult/m6A_abundance/peaks/hg19_TSS/bamCov/adult")
count_list = glob.glob("*.count")
tissue_list = list(set([x.split("_")[0] for x in count_list]))
print(tissue_list)

['brain', 'lung', 'stomach', 'heart', 'liver']


In [3]:
tss_dict = {}
with open("contain_a_TSS.bed", 'r') as f:
    contents = f.readlines()
    for i in range(len(contents)):
        tss_dict[i] = contents[i].strip()

In [5]:
def select_tss(in_file):
    row_nums = []
    with open(in_file, 'r') as f:
        lines = f.readlines()
        for i in range(len(lines)):
            num_list = [int(x) for x in lines[i].strip().split("\t")[6:]]
            edge_index = int(len(num_list)/2)
            num_ip = len([x for x in num_list[:edge_index] if x >= 5])
            num_input = len([x for x in num_list[edge_index:] if x >= 5])
            if (num_ip >= np.ceil(edge_index/2)) and (num_input >= np.ceil(edge_index/2)):
                row_nums.append(i)
    return row_nums

for tissue in tissue_list:
    print(tissue)
    left_count, right_count = "%s_left.count" % tissue, "%s_right.count" % tissue
    left_nums, right_nums = select_tss(left_count), select_tss(right_count)
    total_nums = list(set(left_nums + right_nums))
    with open("%s_selected_TSS.bed" % tissue, 'w') as fw:
        for x in total_nums:
            fw.write(tss_dict[x] + "\n")

brain
lung
stomach
heart
liver


In [6]:
### overlap with m6a peak
tss_list = glob.glob("*_selected_TSS.bed")
for tss in tss_list:
    tissue = tss.split("_")[0]
    m6a = "/Data_Resource/m6A-seq/com_fetus_vs_adult/m6A_abundance/peaks/adult/combined_peaks/by_sample/%s.bed" % tissue
    m6am = "%s_m6am.bed" % tissue 
    os.system("bedtools intersect -a %s -b %s -wa > %s" % (m6a, tss, m6am))

In [7]:
### select m6a from raw m6a peak
!ls *_m6am.bed | cut -d "_" -f1 | parallel "grep -v -f {}_m6am.bed /Data_Resource/m6A-seq/com_fetus_vs_adult/m6A_abundance/peaks/adult/combined_peaks/by_sample/{}.bed > {}_m6a.bed"

Academic tradition requires you to cite works you base your article on.
When using programs that use GNU Parallel to process data for publication
please cite:

  O. Tange (2011): GNU Parallel - The Command-Line Power Tool,
  ;login: The USENIX Magazine, February 2011:42-47.

This helps funding further development; AND IT WON'T COST YOU A CENT.
If you pay 10000 EUR you should feel free to use GNU Parallel without citing.

To silence this citation notice: run 'parallel --citation'.

